In [ ]:
import numpy as np
from ml.algorithms.optimization import GradientDescent, StochasticGradientDescent
from ml.algorithms.normalization import MinMaxScaler
from ml.models.linear import PolynomialRegression, LinearRegression

In [ ]:
import numpy as np

from ml.model_selection import GridSearchCV
from ml.models.linear import PolynomialRegression

from ml.algorithms.normalization import MinMaxScaler
from ml.algorithms.optimization import GradientDescent, OrdinaryLeastSquares

grid = GridSearchCV()

for p in [3, 2, 1]:
    for lr in [0.1, 0.05]:
        grid.add(
            PolynomialRegression, {
                "solver": GradientDescent(epochs=4000, learning_rate=lr, regularization=0.001), 
                "degree": p,
                "dataScaler": MinMaxScaler
            }
        )

    # grid.add(
    #     PolynomialRegression, {
    #         "solver": OrdinaryLeastSquares(regularization=0.0), 
    #         "degree": p,
    #         "dataScaler": MinMaxScaler
    #     }
    # )

best = grid.search(X_train, y_train, num_folds=4, shuffle=True)
print("Best model:", best)
print("Learning rate:", best[1]['solver'].learning_rate)

In [ ]:
from ml.stats import rmse

model = best[0]
model_args = best[1]
poly = model(**model_args)

poly.fit(X_train, y_train)
print(rmse(y_test, poly.predict(X_test)))

In [ ]:
for i in range(len(grid.candidates)):
    model = grid.candidates[i][0]
    model_args = grid.candidates[i][1]
    poly = model(**model_args)

    poly.fit(X_train, y_train)
    print(rmse(y_test, poly.predict(X_test)))

## Logistic regression

In [ ]:
from ml.dataset import load_csv, split_train_test

from ml.models.linear import LogisticRegression
from ml.algorithms.normalization import MinMaxScaler
from ml.algorithms.optimization import GradientDescent

from ml.functions.metrics.classification import BinaryAccuracy, F1Score

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(25)

In [ ]:
X, y = load_csv('./datasets/breastcancer.csv')
X_train, y_train, X_test, y_test = split_train_test(X, y, 0.8, shuffle=True)

In [ ]:
epochs = 5000
learning_rate = 0.05
regularization = 0.0

gd = GradientDescent(epochs, learning_rate, regularization, metrics=BinaryAccuracy())

In [ ]:
lr = LogisticRegression(gd, dataScaler=MinMaxScaler)
train_acc = lr.fit(X_train, y_train)

plt.plot(train_acc)
plt.show()

In [ ]:
y_hat = lr.predict(X_test)
test_accuracy = BinaryAccuracy().measure(y_test, y_hat)
print(test_accuracy)

In [ ]:
y_hat_rounded = np.round(y_hat)
f1 = F1Score().measure(y_test, y_hat_rounded)
print(f1)